In [1]:
'''
Load the necessary libraries
'''
from mpl_toolkits.mplot3d import Axes3D 
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.graph_objs import *
import plotly.express as px
import seaborn as sns
%matplotlib inline
sns.set_context('poster')
sns.set_style('white')
sns.set_color_codes()
plot_kwds = {'alpha' : 0.5, 's' : 80, 'linewidths':0}

import hdbscan

import os
import glob

from sklearn.neighbors import NearestNeighbors
from numpy import linalg as LA
import numpy as np
import pandas as pd

In [2]:
#def cluster_nuclei(filename,sample_size,n_neighbors,threshold_q,min_cluster_size,min_samples):
def cluster_nuclei(filename,sample_size,n_neighbors,threshold_q,auto_open):
    df = pd.read_pickle(filename)
    if sample_size > 0 and sample_size < df.shape[0]:
        df = df.sample(n=sample_size)
    embedding = df[['x','y','z']].to_numpy()
    
    '''
    Calculate the local curvature of the point cloud embedding
    '''
    # Find the neighborood
#    n_neighbors = 100 # parameter to be adjausted

    nbrs = NearestNeighbors(n_neighbors=n_neighbors, algorithm='kd_tree').fit(embedding)
    distances, indices = nbrs.kneighbors(embedding)
    eigvals = [LA.eigvalsh(np.cov(embedding[indices[idx,:],:].T)) for idx in range(embedding.shape[0])] #full data

    curvatures = [min(eigvals[idx])/sum(eigvals[idx]) for idx in range(len(eigvals))]

    # Add curvature to the dataframe
    df['curvature'] = curvatures 

    # Find the minima in curvature histrogram
    q1 = np.quantile(curvatures,threshold_q)

    df1 = df[df['curvature'] <= q1] # define the low curvature sector

    min_cluster_size = round(df1.shape[0]/15) # parameter to be adjausted
    min_samples = round(min_cluster_size/15)       # parameter to be adjausted

    clusterer = hdbscan.HDBSCAN(min_samples=min_samples,min_cluster_size=min_cluster_size,gen_min_span_tree=True)
    clusterer.fit(df1.loc[:,('x','y','z')]) 

    clusterer.condensed_tree_.plot(select_clusters=True,
                                   selection_palette=sns.color_palette("Set2",len(clusterer.labels_)))
    plt.savefig(filename+'.tree.png')
    plt.close()
    
    df1['cluster'] = clusterer.labels_    # add cluster id to dataframe
    df1['cluster'] = df1['cluster'].apply(str)   # make cluster id a string
    df_filtered = df1[df1.cluster != str(-1)] # remove unassigned points

    fig = px.scatter(df_filtered, x="cx", y="cy",color="cluster",
                         width=800, height=800,
                         color_discrete_sequence=px.colors.qualitative.Set2)
    fig.update_traces(marker=dict(size=5,opacity=1.0))
#    fig.write_html(filename+'.spatial_decoration.html', auto_open=auto_open)
    fig.write_image(filename+'.spatial_decoration.png')

    fig = px.scatter_3d(df_filtered, x="x", y="y", z="z", color="cluster", hover_name="cluster", 
                            color_discrete_sequence=px.colors.qualitative.Set2)
    fig.update_traces(marker=dict(size=3,opacity=0.75),selector=dict(mode='markers'))
#    fig.write_html(filename+'.low_curvature_clusters.html', auto_open=auto_open)
    fig.write_image(filename+'.low_curvature_clusters.png')
    df1.to_csv(filename+'.csv',index=False)
    return df1

In [72]:
#def cluster_nuclei(filename,sample_size,n_neighbors,threshold_q,min_cluster_size,min_samples):
def cluster_nuclei(filename,sample_size,n_neighbors,threshold_q,auto_open):
    df = pd.read_pickle(filename)
    if sample_size > 0 and sample_size < df.shape[0]:
        df = df.sample(n=sample_size)
    embedding = df[['x','y','z']].to_numpy()
    
    '''
    Calculate the local curvature of the point cloud embedding
    '''
    # Find the neighborood
#    n_neighbors = 100 # parameter to be adjausted

    nbrs = NearestNeighbors(n_neighbors=n_neighbors, algorithm='kd_tree').fit(embedding)
    distances, indices = nbrs.kneighbors(embedding)
    eigvals = [LA.eigvalsh(np.cov(embedding[indices[idx,:],:].T)) for idx in range(embedding.shape[0])] #full data

    curvatures = [min(eigvals[idx])/sum(eigvals[idx]) for idx in range(len(eigvals))]

    # Add curvature to the dataframe
    df['curvature'] = curvatures 

    # Find the minima in curvature histrogram
    q1 = np.quantile(curvatures,threshold_q)

    df1 = df[df['curvature'] <= q1] # define the low curvature sector

    min_cluster_size = round(df1.shape[0]/15) # parameter to be adjausted
    min_samples = round(min_cluster_size/15)       # parameter to be adjausted

    clusterer = hdbscan.HDBSCAN(min_samples=min_samples,min_cluster_size=min_cluster_size,gen_min_span_tree=True)
    clusterer.fit(df1.loc[:,('x','y','z')]) 

    clusterer.condensed_tree_.plot(select_clusters=True,
                                   selection_palette=sns.color_palette("Set2",len(clusterer.labels_)))
    plt.savefig(filename+'.tree.png')
    plt.close()
    df1['cluster'] = clusterer.labels_    # add cluster id to dataframe
    df1['cluster'] = df1['cluster'].apply(str)   # make cluster id a string
    df1.to_csv(filename+'.csv',index=False)
    return df1

In [3]:
sample_size = 100000
n_neighbors = 100
threshold_q = 0.1
#min_cluster_size = 1000
#min_samples = 500

In [4]:
for filename in glob.glob(r'../pkl/*.pkl'):
    #df_out = cluster_nuclei(filename,sample_size,n_neighbors,threshold_q,min_cluster_size,min_samples)
    df_out = cluster_nuclei(filename,sample_size,n_neighbors,threshold_q,auto_open=False)

/home/garner1/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/garner1/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:40: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/garner1/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

/home/garner1/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/garner1/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:40: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [71]:
df_out

,fov_row,fov_col,cx,cy,x,y,z,area,perimeter,solidity,eccentricity,mean_intensity,curvature
161560,47,25,13162.564103,24562.512821,-4.479586,-5.361786,4.985491,39.0,22.485281,0.928571,0.546823,3.820513,0.110089
366598,44,32,16818.298851,22825.839080,-3.417250,-3.345840,8.666857,87.0,33.798990,0.945652,0.739355,2.045977,0.000828
204034,30,41,21439.097087,15866.058252,7.325370,-5.344365,-0.762070,103.0,46.071068,0.811024,0.681481,8.116505,0.084650
421445,34,54,27954.650943,17820.476415,-5.000680,-0.071517,-1.778364,212.0,78.254834,0.806084,0.924956,3.084906,0.109150
349116,35,26,13371.587638,18257.546147,6.400230,-6.501708,-3.835138,1181.0,181.095454,0.827610,0.555947,9.811177,0.031829
...,...,...,...,...,...,...,...,...,...,...,...,...,...
304639,11,40,20872.552885,6137.312500,7.190392,-4.085598,1.837913,208.0,71.698485,0.845528,0.868784,5.812500,0.113157
33583,18,18,9661.039216,9438.156863,-0.161852,1.859574,-3.844137,51.0,24.727922,0.962264,0.663288,7.901961,0.109288
174228,21,54,27712.503614,11220.939759,-5.389044,-1.490590,0.760269,415.0,102.325902,0.788973,0.842133,2.000000,0.076360
60904,36,17,9035.657143,18897.828571,-3.190760,-3.461745,9.113662,35.0,20.449747,0.875000,0.551628,2.028571,0.000944
